In [3]:
# !pip install folium
#!pip install utm
#!pip install shapely

In [2]:
from h3 import h3
import utm
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
path = r'C:\Users\NiFa\Desktop\Data Science\CapStone\data\Ride_Data_Cache'
order_data = pd.read_msgpack('/mnt/c/Ride_Data_Cache/order_20161102.msgpack')

In [5]:
gps_data = pd.read_msgpack('/mnt/c/Ride_Data_Cache/gps_20161102.msgpack')

In [6]:
drivers = gps_data[['driver_id', 'order_id']].drop_duplicates()
order_data  = order_data.merge(drivers, on ='order_id', how='left')

In [7]:
order_data.head()

,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,driver_id
0,39d471630d26439111a7746d164d34eb,1478091677,1478092890,104.008160,30.706220,104.064147,30.685848,af17fba73f06db61f7596163eac1eb73
1,6681082cbcc11b37842489a096e2172e,1478093722,1478094806,104.053993,30.684104,104.015670,30.633230,af17fba73f06db61f7596163eac1eb73
2,56728c3d9555a452307d47986a773d9d,1478081351,1478082003,104.068120,30.708790,104.049720,30.704970,ae8221abe491455c7c0bedb0f00d6e8a
3,5b2724a9fd290d0388264fe07a9aec4c,1478082542,1478083418,104.059534,30.696013,104.034630,30.720960,ae8221abe491455c7c0bedb0f00d6e8a
4,0a2055d9c863e2f1963856d4d305ccd9,1478078909,1478080807,104.079048,30.796888,104.083284,30.705019,ae8221abe491455c7c0bedb0f00d6e8a


In [8]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)   
from h3 import h3
import folium

def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m
    

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

In [9]:
minlat = min(order_data.pickup_latitude)
minlong = min(order_data.pickup_longitude)
maxlat = max(order_data.pickup_latitude)
maxlong = max(order_data.pickup_longitude)

In [22]:
geoJson = {'type': 'Polygon',
  'coordinates': [[[minlat, minlong], [minlat, maxlong ], [maxlat, maxlong], [ maxlat, minlong ]]] }

In [23]:
# geoJson = {'type': 'Polygon',
#  'coordinates': [[[37.813318999983238, -122.4089866999972145], [ 37.7866302000007224, -122.3805436999997056 ], [37.7198061999978478, -122.3544736999993603], [ 37.7076131999975672, -122.5123436999983966 ], [37.7835871999971715, -122.5247187000021967],  [37.8151571999998453, -122.4798767000009008]]] }
res = 7
polyline = geoJson['coordinates'][0]
polyline.append(polyline[0])
lat = [p[0] for p in polyline]
lng = [p[1] for p in polyline]
m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
my_PolyLine=folium.PolyLine(locations=polyline,weight=4,color="green")
m.add_child(my_PolyLine)

hexagons = list(h3.polyfill(geoJson, res))
polylines = []
lat = []
lng = []
for hex in hexagons:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    lat.extend(map(lambda v:v[0],polyline))
    lng.extend(map(lambda v:v[1],polyline))
    polylines.append(polyline)
for polyline in polylines:
    my_PolyLine=folium.PolyLine(locations=polyline,weight=1,color='red')
    m.add_child(my_PolyLine)
display(m)
m.save("hex_grid_" + str(res) + ".html")

In [13]:
# m.save("hex_grid_6.html")

In [26]:
polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
a= np.array(polygons[0][0])

In [27]:
b = utm.from_latlon(a[:,0],a[:,1])

In [28]:
xy_pickup = utm.from_latlon(order_data.pickup_latitude.values,order_data.pickup_longitude.values)
x_pickup = list(xy_pickup[0])
y_pickup = list(xy_pickup[1])
pickup_point = list(zip(x_pickup, y_pickup))


In [38]:
poly_hex = dict()
for i,hex in enumerate(hexagons):
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    a= np.array(polygons[0][0])
    b = utm.from_latlon(a[:,0],a[:,1])
    poly_hex[i] = Polygon(list(zip(b[0], b[1]))) 

In [34]:
# pick_zone = np.zeros(len(order_data))-1

# for j,p in enumerate(pickup_point):
#     point = Point(p)
#     for polygon in range(len(poly_hex)):
        
#         if polygon.contains(point):
#             pick_zone[j] = int(i)
#             break
    

# order_data['pickup_zone'] = pick_zone

In [40]:
pick_zone = np.zeros(len(order_data))-1

for j,p in enumerate(pickup_point):
    point = Point(p)
    for i in range(len(poly_hex)):
        polygon = poly_hex[i]
        
        if polygon.contains(point):
            pick_zone[j] = int(i)
            break
    

order_data['pickup_zone'] = pick_zone

In [ ]:
get_spatial_features_hex(df, resolution =6, use_cache=True)

In [ ]:
temp = order_data.groupby(['driver_id', 'pickup_zone']).count()/ order_data.groupby(['driver_id']).count()

In [ ]:
temp